# Data Wrangling

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#from pandas_profiling import ProfileReport
%matplotlib inline

In [2]:
pwd

'D:\\Jupyter_Notebook\\Movie_Recommendation_System'

## Load the data
There are 3 datasets: movies, ratings, and users.
These files contain 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens in 2000.

In [3]:
# Define file directories
file_dir = 'DAT Files'
user_dat = 'users.dat'
movie_dat = 'movies.dat'
rating_dat = 'ratings.dat'

### Ratings
All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

In [4]:
# Read the Ratings File
ratings = pd.read_csv(os.path.join(file_dir, rating_dat), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(len(ratings), 'ratings loaded')

1000209 ratings loaded


In [5]:
ratings.shape

(1000209, 4)

In [6]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


### Users
User information is in the file "users.dat" and is in the following
format:
UserID::Gender::Age::Occupation::Zip-code

All demographic information is provided voluntarily by the users and is
not checked for accuracy.  Only users who have provided some demographic
information are included in this data set.

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- Occupation is chosen from the following choices:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
- zipcode is the demographic information of this user

In [8]:
AGES = { 1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44", 45: "45-49", 50: "50-55", 56: "56+" }
OCCUPATIONS = { 0: "other or not specified", 1: "academic/educator", 2: "artist", 3: "clerical/admin",
                4: "college/grad student", 5: "customer service", 6: "doctor/health care",
                7: "executive/managerial", 8: "farmer", 9: "homemaker", 10: "K-12 student", 11: "lawyer",
                12: "programmer", 13: "retired", 14: "sales/marketing", 15: "scientist", 16: "self-employed",
                17: "technician/engineer", 18: "tradesman/craftsman", 19: "unemployed", 20: "writer" }

In [9]:
# Read the Users File
users = pd.read_csv(os.path.join(file_dir, user_dat), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['user_id', 'gender', 'age', 'occupation', 'zipcode'])

# Create a column called age_desc to represent a hunman readable age range
users['age_desc'] = users['age'].apply(lambda x: AGES[x])

# Create a column called occ_desc to represent a hunman readable occupation
users['occ_desc'] = users['occupation'].apply(lambda x: OCCUPATIONS[x])
print(len(users), 'users loaded.')

6040 users loaded.


In [10]:
users.shape

(6040, 7)

In [11]:
users.head()

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc
0,1,F,1,10,48067,Under 18,K-12 student
1,2,M,56,16,70072,56+,self-employed
2,3,M,25,15,55117,25-34,scientist
3,4,M,45,7,02460,45-49,executive/managerial
4,5,M,25,20,55455,25-34,writer


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zipcode     6040 non-null   object
 5   age_desc    6040 non-null   object
 6   occ_desc    6040 non-null   object
dtypes: int64(3), object(4)
memory usage: 330.4+ KB


### Movies
- MovieIDs range between 1 and 3952
- Titles are identical to titles provided by the IMDB (including
year of release)
- Genres are pipe-separated and are selected from the following genres:

	1. Action
	2. Adventure
	3. Animation
	4. Children's
	5. Comedy
	6. Crime
	7. Documentary
	8. Drama
	9. Fantasy
	10. Film-Noir
	11. Horror
	12. Musical
	13. Mystery
	14. Romance
	15. Sci-Fi
	16. Thriller
	17. War
	18. Western

- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist


In [13]:
# Read the Movies File
movies = pd.read_csv(os.path.join(file_dir, movie_dat), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])

In [14]:
movies.shape

(3883, 3)

In [15]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genres    3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


## Look for duplicate rows

In [17]:
print("There are {0} duplicate rows in ratings".format(ratings.duplicated().sum()))
print("There are {0} duplicate rows in users".format(users.duplicated().sum()))
print("There are {0} duplicate rows in movies".format(movies.duplicated().sum()))

There are 0 duplicate rows in ratings
There are 0 duplicate rows in users
There are 0 duplicate rows in movies


## Missing Values

In [18]:
# Check for missing values in ratings -- no missing value
print(ratings.isnull().sum())

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64


In [19]:
# Check for missing values in users -- no missing value
print(users.isnull().sum())

user_id       0
gender        0
age           0
occupation    0
zipcode       0
age_desc      0
occ_desc      0
dtype: int64


In [20]:
# Check for missing values in movies -- no missing value
print(movies.isnull().sum())

movie_id    0
title       0
genres      0
dtype: int64


## Data description

In [21]:
# Check if there are any incorrect information. Focusing on min and max in this table
# There are 3,883 movies made by 6,040 MovieLens users
print(ratings.describe().T)
print(movies['movie_id'].nunique())

               count          mean           std          min          25%  \
user_id    1000209.0  3.024512e+03  1.728413e+03          1.0       1506.0   
movie_id   1000209.0  1.865540e+03  1.096041e+03          1.0       1030.0   
rating     1000209.0  3.581564e+00  1.117102e+00          1.0          3.0   
timestamp  1000209.0  9.722437e+08  1.215256e+07  956703932.0  965302637.0   

                   50%          75%           max  
user_id         3070.0       4476.0  6.040000e+03  
movie_id        1835.0       2770.0  3.952000e+03  
rating             4.0          4.0  5.000000e+00  
timestamp  973018006.0  975220939.0  1.046455e+09  
3883


In [22]:
# There are 3,883 movies, nothing seems to be incorrect
print(movies.describe().T)
print(movies.describe(include=['object', 'category']).T)
print(movies['movie_id'].nunique())

           count         mean          std  min    25%     50%     75%     max
movie_id  3883.0  1986.049446  1146.778349  1.0  982.5  2010.0  2980.5  3952.0
       count unique                     top freq
title   3883   3883  Lethal Weapon 4 (1998)    1
genres  3883    301                   Drama  843
3883


In [23]:
# There are 6,040 users, 7 age ranges,21 occupations. Nothing seems to be incorrect
print(users.describe().T)
print(users.describe(include=['object', 'category']).T)

             count         mean          std  min      25%     50%      75%  \
user_id     6040.0  3020.500000  1743.742145  1.0  1510.75  3020.5  4530.25   
age         6040.0    30.639238    12.895962  1.0    25.00    25.0    35.00   
occupation  6040.0     8.146854     6.329511  0.0     3.00     7.0    14.00   

               max  
user_id     6040.0  
age           56.0  
occupation    20.0  
         count unique                   top  freq
gender    6040      2                     M  4331
zipcode   6040   3439                 48104    19
age_desc  6040      7                 25-34  2096
occ_desc  6040     21  college/grad student   759


## Convert timestamp to datetime

In [32]:
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [34]:
ratings.drop(['timestamp'],axis=1, inplace=True)

In [35]:
ratings.head()

,user_id,movie_id,rating,datetime
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


## Save data into csv files

In [39]:
# Save into ratings.csv
ratings.to_csv(os.path.join('data', 'ratings.csv'), 
               header=True, 
               encoding='latin-1', 
               columns=['user_id', 'movie_id', 'rating', 'datetime'])

In [40]:
# Save into users.csv
users.to_csv(os.path.join('data', 'users.csv'),
             header=True, 
             encoding='latin-1',
             columns=['user_id', 'gender', 'age', 'occupation', 'zipcode', 'age_desc', 'occ_desc'])

In [41]:
# Save into movies.csv
movies.to_csv(os.path.join('data', 'movies.csv'), 
              header=True, 
              columns=['movie_id', 'title', 'genres'])